# Train model

In [1]:
from fbprophet import Prophet
import pandas as pd
import os
from joblib import dump, load
import datetime
from pathlib import Path
import json
from fbprophet.serialize import model_to_json, model_from_json

In [2]:
DATA_DIR = "data"
INPUT_DIRNAME = "02_clean"
INPUT_FILENAME = "clean_data.csv"
OUTPUT_DIRNAME = "artifacts"
OUTPUT_FILENAME = "prophet.json"
EXECUTION_DATE = pd.to_datetime("2021-03-18")
DATE_FORMAT = "%Y-%m-%d"

In [3]:
OUTPUT_DIR = Path(os.path.join(OUTPUT_DIRNAME, EXECUTION_DATE.strftime(DATE_FORMAT)))
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [4]:
def load_all_available_data_before_date(path: str) -> pd.DataFrame:
    dataframes = [pd.read_csv(os.path.join(path, name, INPUT_FILENAME)) for name in os.listdir(path) if "." not in name and pd.to_datetime(name) <= EXECUTION_DATE]
    return pd.concat(dataframes, ignore_index=True)

In [5]:
# Load data
data = load_all_available_data_before_date(os.path.join(DATA_DIR, INPUT_DIRNAME))
data = data.rename(columns={"Date": "ds", "Close": "y"}).sort_values("ds").reset_index(drop=True)
data

,ds,y
0,2021-01-01,29374.152344
1,2021-01-02,32127.267578
2,2021-01-03,32782.023438
3,2021-01-04,31971.914062
4,2021-01-05,33992.429688
...,...,...
72,2021-03-14,59844.519531
73,2021-03-15,55907.199219
74,2021-03-16,56304.406250
75,2021-03-17,55904.730469


In [6]:
model = Prophet()
model.fit(data)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [7]:
with open(os.path.join(OUTPUT_DIR, OUTPUT_FILENAME), 'w') as smodel:
    json.dump(model_to_json(model), smodel)